In [1]:
import speech_recognition as sr
import spacy  # An NLP tool similar to NLTK
from datetime import datetime
from spacy.matcher import PhraseMatcher, Matcher
from gtts import gTTS 
from playsound import playsound

nlp = spacy.load('en_core_web_sm')  # A dictionary used for NLP tasks

In [2]:
import os 

In [ ]:
var = 1 # infinite loop
language = 'en' # Language in which you want to convert 

while var == 1 :
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak!")
        audio = r.listen(source)
    
    direction_matcher = PhraseMatcher(nlp.vocab)  # A matcher that matches directions
    terms = ['forward', 'backward']
    # Only run nlp.make_doc to speed things up
    patterns = [nlp.make_doc(text) for text in terms]
    direction_matcher.add("TerminologyList", None, *patterns)

    distance_matcher = Matcher(nlp.vocab)  # A matcher that matches cm
    distance_pattern = [{"IS_DIGIT": True}, {"LOWER": "cm"}]
    distance_matcher.add("Distance", None, distance_pattern)
    
    rotation_matcher = Matcher(nlp.vocab)
    rotation_pattern = [{"IS_DIGIT": True}, {"LOWER": "degrees"}]  # A matcher that matches degrees
    rotation_matcher.add("Rotation", None, rotation_pattern)
    
    audio = r.recognize_google(audio)
    print("Google Speech Recognition thinks you said: " + audio)
    
    while (audio == "robot") :  # Using "robot" to activate because "little robot" is often misrecognized
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Robot activated!")
            audio = r.listen(source)
        audio = r.recognize_google(audio)
        print("Google Speech Recognition thinks you said: " + audio)
        doc = nlp(audio)  # Process the sentence
        
        direction_matches = direction_matcher(doc)  # Match the pattern "forward" or "backward"
        for match_id, start, end in direction_matches:
            direction = doc[start:end]  # Get the desired direction
            if (direction.text == 'forward'):
                distance_matches = distance_matcher(doc)  # Match the pattern "xx cm"
                for match_id, start, end in distance_matches:
                    distance = doc[start:end]  # Get the desired distance
                    print('Advance confirmed', distance.text)
                    mytext = 'OK advance' + distance.text
                    myobj = gTTS(text=mytext, lang=language, slow=False)  # Text to speech
                    # Saving the converted audio in a mp3 file named 
                    myobj.save('advance.mp3') 
                    playsound('advance.mp3') 
                    os.remove('advance.mp3')  # Had to remove the file otherwise there will be a permission problem

            elif (direction.text == 'backward'):
                distance_matches = distance_matcher(doc)  # Match the pattern "xx cm"
                for match_id, start, end in distance_matches:
                    distance = doc[start:end]  # Get the desired distance
                    print('Retreat confirmed', distance.text)
                    mytext = 'OK retreat' + distance.text
                    myobj = gTTS(text=mytext, lang=language, slow=False)  # Text to speech
                    # Saving the converted audio in a mp3 file named 
                    myobj.save('retreat.mp3') 
                    playsound('retreat.mp3') 
                    os.remove('retreat.mp3')  # Had to remove the file otherwise there will be a permission problem
                    
        rotate_matches = rotation_matcher(doc)  # Match the pattern "xx degrees"
        for match_id, start, end in rotate_matches:
            rotation = doc[start:end]  # Get the desired rotation
            print('Rotate confirmed', rotation.text)
            mytext = 'OK rotate' + rotation.text
            myobj = gTTS(text=mytext, lang=language, slow=False)  # Text to speech
            # Saving the converted audio in a mp3 file named 
            myobj.save('rotation.mp3') 
            playsound('rotation.mp3') 
            os.remove('rotation.mp3')  # Had to remove the file otherwise there will be a permission problem
    
    ############ end of while loop #############

## There is a problem with TTS, it only works for the first voice command. It seems it may not be a good idea to delete
## the audio file but I did not find a better solution.